In [1]:
import math
import torch
# import gpytorch
import gpytorch 
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2
# Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 0.939   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.908   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.874   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.836   lengthscale: 0.555   noise: 0.554
Iter 5/50 - Loss: 0.793   lengthscale: 0.514   noise: 0.513
Iter 6/50 - Loss: 0.745   lengthscale: 0.476   noise: 0.474
Iter 7/50 - Loss: 0.692   lengthscale: 0.439   noise: 0.437
Iter 8/50 - Loss: 0.639   lengthscale: 0.405   noise: 0.402
Iter 9/50 - Loss: 0.588   lengthscale: 0.372   noise: 0.369
Iter 10/50 - Loss: 0.542   lengthscale: 0.342   noise: 0.339
Iter 11/50 - Loss: 0.499   lengthscale: 0.315   noise: 0.310
Iter 12/50 - Loss: 0.461   lengthscale: 0.292   noise: 0.284
Iter 13/50 - Loss: 0.425   lengthscale: 0.272   noise: 0.259
Iter 14/50 - Loss: 0.389   lengthscale: 0.256   noise: 0.236
Iter 15/50 - Loss: 0.355   lengthscale: 0.242   noise: 0.215
Iter 16/50 - Loss: 0.320   lengthscale: 0.232   noise: 0.196
Iter 17/50 - Loss: 0.286   length

In [6]:
train_y

tensor([ 0.1472,  0.0542, -0.0017,  0.0900,  0.0231,  0.1515,  0.5419,  0.4009,
         0.6882,  0.5262,  0.4167,  0.8300,  0.8095,  0.9544,  0.6399,  0.7341,
         0.8128,  0.6992,  1.2033,  1.2183,  0.8351,  1.4617,  0.8766,  0.7882,
         1.2723,  0.5667,  0.9836,  1.2886,  0.7919,  1.1668,  0.9878,  0.7267,
         0.6692,  0.7474,  0.7329,  1.0463,  0.7085,  0.6536,  0.5996,  0.3889,
         0.5397,  0.7419,  0.3112,  0.1850,  0.1799,  0.4450,  0.2444,  0.1980,
        -0.0764, -0.0387,  0.0987,  0.1960, -0.2307, -0.3933, -0.2622, -0.4323,
        -0.4140, -0.5233, -0.6807, -0.6018, -0.6416, -0.5900, -0.9218, -0.2924,
        -0.7890, -0.6631, -0.9443, -0.9310, -1.0282, -1.3754, -1.0006, -1.0264,
        -1.0275, -1.0717, -1.1639, -1.1213, -0.7738, -0.8556, -1.1021, -0.8744,
        -0.7306, -0.6678, -0.6980, -0.9149, -1.1042, -0.5735, -1.0065, -0.6157,
        -0.2069, -0.2999, -0.3711, -0.5814, -0.4191, -0.4433, -0.2388, -0.3950,
         0.0258, -0.2587, -0.2311,  0.26